In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Multiple_sclerosis"
cohort = "GSE189788"

# Input paths
in_trait_dir = "../../input/GEO/Multiple_sclerosis"
in_cohort_dir = "../../input/GEO/Multiple_sclerosis/GSE189788"

# Output paths
out_data_file = "../../output/preprocess/Multiple_sclerosis/GSE189788.csv"
out_gene_data_file = "../../output/preprocess/Multiple_sclerosis/gene_data/GSE189788.csv"
out_clinical_data_file = "../../output/preprocess/Multiple_sclerosis/clinical_data/GSE189788.csv"
json_path = "../../output/preprocess/Multiple_sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Normalization of Large Scale Transcriptome Data Using Heuristic Methods"
!Series_summary	"We introduce AI method for addressing batch effect of genetic data"
!Series_summary	"The method does not rely on any assumptions regarding the distribution and the behavior of data elements. Hence, it does not introduce any new biases in the process of correcting for batch effect. It strictly maintains the integrity of measurements within the original batches."
!Series_overall_design	"The human peripheral mononuclear blood cells (PBMC) samples from multiple sclerosis patients treated in Multiple Sclerosis Center, Sheba Medical Center, Israel were processed on Affymetrix HU-133A-2 microarrays (Santa Clara, CA) according to manufacturer protocol between 2006 and 2014 . A total 216 samples from 216 distinct individuals, combined from 24 batches associated with working sets consisting of more than seven samples of each were analyzed."
Sample Characteristics Dicti

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the description, this dataset uses Affymetrix HU-133A-2 microarrays which contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait: Multiple Sclerosis - All patients have MS as seen in key 0
trait_row = 0

# Age: Available in key 2
age_row = 2

# Gender: Available in key 3
gender_row = 3

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait value to binary (all patients have MS in this dataset)"""
    if 'multiple sclerosis' in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric value"""
    try:
        # Extract the age value after the colon
        age_str = value.split(':')[1].strip()
        return float(age_str)
    except (IndexError, ValueError):
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if ':' in value:
        gender = value.split(':')[1].strip().lower()
        if 'female' in gender:
            return 0
        elif 'male' in gender:
            return 1
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We need to check if a clinical_data DataFrame object is already available in the workspace
    # If not, we can create it from the sample characteristics information
    try:
        # Try to access clinical_data
        clinical_data
    except NameError:
        # Create clinical data from the sample characteristics dictionary
        print("Creating clinical data from sample characteristics dictionary")
        sample_chars = {0: ['patient diagnosis: multiple sclerosis'], 
                        1: ['tissue: PBMC'], 
                        2: ['age(years): 43', 'age(years): 21', 'age(years): 29', 'age(years): 41', 'age(years): 36', 'age(years): 28', 'age(years): 33', 'age(years): 48', 'age(years): 22', 'age(years): 19', 'age(years): 15', 'age(years): 39', 'age(years): 45', 'age(years): 35', 'age(years): 44', 'age(years): 47', 'age(years): 31', 'age(years): 57', 'age(years): 46', 'age(years): 52', 'age(years): 54', 'age(years): 32', 'age(years): 14', 'age(years): 30', 'age(years): 38', 'age(years): 25', 'age(years): 24', 'age(years): 55', 'age(years): 51', 'age(years): 26'], 
                        3: ['gender: female', 'gender: male'], 
                        4: ['batch number: 1', 'batch number: 2', 'batch number: 3', 'batch number: 4', 'batch number: 5', 'batch number: 6', 'batch number: 7', 'batch number: 8', 'batch number: 9', 'batch number: 10', 'batch number: 11', 'batch number: 12', 'batch number: 13', 'batch number: 14', 'batch number: 15', 'batch number: 16', 'batch number: 17', 'batch number: 18', 'batch number: 19', 'batch number: 20', 'batch number: 21', 'batch number: 22', 'batch number: 23', 'batch number: 24']}
        
        # Create a DataFrame from the sample characteristics
        clinical_data = pd.DataFrame(sample_chars)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    print("Preview of extracted clinical features:")
    preview = preview_df(selected_clinical_df)
    print(preview)
    
    # Save to CSV
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM5707576': [1.0, 43.0, 0.0], 'GSM5707577': [1.0, 21.0, 1.0], 'GSM5707578': [1.0, 29.0, 1.0], 'GSM5707579': [1.0, 41.0, 0.0], 'GSM5707580': [1.0, 36.0, 0.0], 'GSM5707581': [1.0, 28.0, 1.0], 'GSM5707582': [1.0, 33.0, 1.0], 'GSM5707583': [1.0, 48.0, 0.0], 'GSM5707584': [1.0, 48.0, 0.0], 'GSM5707585': [1.0, 36.0, 0.0], 'GSM5707586': [1.0, 33.0, 1.0], 'GSM5707587': [1.0, 22.0, 0.0], 'GSM5707588': [1.0, 19.0, 1.0], 'GSM5707589': [1.0, 22.0, 1.0], 'GSM5707590': [1.0, 36.0, 0.0], 'GSM5707591': [1.0, 15.0, 0.0], 'GSM5707592': [1.0, 39.0, 0.0], 'GSM5707593': [1.0, 19.0, 0.0], 'GSM5707594': [1.0, 45.0, 1.0], 'GSM5707595': [1.0, 36.0, 0.0], 'GSM5707596': [1.0, 28.0, 0.0], 'GSM5707597': [1.0, 33.0, 0.0], 'GSM5707598': [1.0, 35.0, 0.0], 'GSM5707599': [1.0, 44.0, 0.0], 'GSM5707600': [1.0, 47.0, 0.0], 'GSM5707601': [1.0, 31.0, 0.0], 'GSM5707602': [1.0, 47.0, 0.0], 'GSM5707603': [1.0, 31.0, 1.0], 'GSM5707604': [1.0, 57.0, 0.0], 'GSM5707605': [1.0, 46.0, 1.0],

### Step 3: Gene Data Extraction

In [4]:
# 1. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 3. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')

Gene data dimensions: 22277 genes × 216 samples


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs, not human gene symbols
# Identifiers like "1007_s_at" are characteristic of Affymetrix microarray platforms
# They need to be mapped to proper human gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the gene annotation preview, I can see:
# - 'ID' column in gene_annotation contains probe IDs (like '1007_s_at')
# - 'Gene Symbol' column contains human gene symbols (like 'DDR1 /// MIR4640')

# Get a mapping dataframe with the probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 2. Apply the gene mapping to convert probe-level measurements to gene expression data
# This handles the many-to-many relationships between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_df)

# 3. Print information about the mapped gene expression data
print(f"\nAfter mapping: Gene expression data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("\nFirst 10 gene symbols after mapping:")
print(gene_data.index[:10])



After mapping: Gene expression data dimensions: 13830 genes × 216 samples

First 10 gene symbols after mapping:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB',
       'AAK1', 'AAMDC'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the clinical data that was previously processed
clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
print("Clinical data shape:", clinical_data.shape)

# Fix clinical data format: ensure samples are rows and features are columns
# In GEO data, clinical features are typically rows, so we need to transpose
if 'Multiple_sclerosis' in clinical_data.index:
    clinical_data = clinical_data.T
    print("Transposed clinical data to have samples as rows")
    print("Clinical data after transposition:", clinical_data.shape)

# 3. Link clinical and genetic data
# Need to ensure sample IDs are in the same format
gene_data_samples = normalized_gene_data.columns
clinical_data_samples = clinical_data.index

# Check if sample IDs need alignment
if not any(sample in gene_data_samples for sample in clinical_data_samples):
    # Try to find a common format or pattern in the IDs
    print("Sample ID formats don't match directly, attempting to align...")
    
    # Check if the sample IDs in gene_data start with "GSM"
    if all('GSM' in str(sample) for sample in gene_data_samples):
        # Create a mapping between normalized_gene_data column names and clinical_data index
        gene_data_t = normalized_gene_data.T
        # Ensure index is string type
        gene_data_t.index = gene_data_t.index.astype(str)
        clinical_data.index = clinical_data.index.astype(str)
    else:
        print("Unable to align sample IDs automatically.")

# Create a mapping from clinical data feature names to standard names
# Assuming the clinical data columns are ["Multiple_sclerosis", "Age", "Gender"]
clinical_data = clinical_data.rename(columns={"Multiple_sclerosis": trait})

# Now combine clinical and genetic data - use gene_data_t which has samples as rows
linked_data = pd.concat([clinical_data, normalized_gene_data.T], axis=1)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
print(f"Data shape after removing biased features: {linked_data.shape}")
print(f"Is the trait biased: {is_biased}")

# 6. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from PBMC samples of Multiple Sclerosis patients."
)

# 7. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable. Linked data was not saved.")

Gene data shape after normalization: (13542, 216)
First 5 gene symbols after normalization: Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Multiple_sclerosis/gene_data/GSE189788.csv
Clinical data shape: (3, 216)
Transposed clinical data to have samples as rows
Clinical data after transposition: (216, 3)
Linked data shape before handling missing values: (216, 13545)


Data shape after handling missing values: (216, 13545)
Quartiles for 'Multiple_sclerosis':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Multiple_sclerosis' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 27.75
  50% (Median): 36.0
  75%: 45.0
Min: 8.0
Max: 73.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 72 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data shape after removing biased features: (216, 13545)
Is the trait biased: True
Dataset deemed not usable. Linked data was not saved.
